In [8]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import ViTForImageClassification, ViTFeatureExtractor
from PIL import Image
import pandas as pd
import os

In [9]:
from transformers import ViTForImageClassification, ViTFeatureExtractor

#Load ImageNet pre-trained ViT
model_name = 'google/vit-base-patch16-224-in21k' #Model version
model = ViTForImageClassification.from_pretrained(model_name, num_labels = 8) #CheXray-8

# Feature extractor for preprocessing images
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [ ]:
class CheXrayDataset(Dataset):
    def __init__(self, img_dir, labels_csv, feature_extractor, transform=None):
        self.img_dir = img_dir
        self.labels = pd.read_csv(labels_csv)
        self.feature_extractor = feature_extractor
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        row = self.labels.iloc[idx]
        img_path = os.path.join(self.img_dir, row['Image Index'])
        image = Image.open(img_path).convert('RGB')

        # Apply preprocessing
        inputs = self.feature_extractor(images=image, return_tensors="pt")

        # Convert label row to tensor
        label = torch.tensor(row[1:].values.astype(float))  # assuming labels are from column 1 onwards (multi-label)

        return {
            'pixel_values': inputs['pixel_values'].squeeze(),  # remove batch dim
            'labels': label
        }


In [ ]:
model_name = 'google/vit-base-patch16-224-in21k'
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)

model = ViTForImageClassification.from_pretrained(
    model_name,
    num_labels=8,       # 8 diseases
    problem_type="multi_label_classification"  # Important for CheXray8
)

In [ ]:
dataset = CheXrayDataset(
    img_dir='/path/to/images',
    labels_csv='/path/to/labels.csv',
    feature_extractor=feature_extractor
)

dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.BCEWithLogitsLoss()  # for multi-label classification

epochs = 100  # adjust as needed

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in dataloader:
        inputs = batch['pixel_values'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(pixel_values=inputs)
        logits = outputs.logits

        loss = criterion(logits, labels)
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(dataloader)}")
